In [81]:
import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings(action="ignore")

In [8]:
os.listdir('iot_data/datasets/dataset_ML_DL')

['ML-EdgeIIoT-dataset.csv', 'DNN-EdgeIIoT-dataset.csv']

In [257]:
df = pd.read_csv('iot_data/datasets/dataset_ML_DL/ML-EdgeIIoT-dataset.csv')
# df = pd.read_csv('iot_data/datasets/normal_traffic/Distance/Distance.csv')

In [34]:
len(df[df.Attack_label == 1])/len(df)

0.8460012674271229

In [35]:
len(df[df.Attack_label == 0])/len(df)

0.15399873257287705

In [33]:
df.describe()

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.transmit_timestamp,icmp.unused,http.content_length,http.response,http.tls_port,tcp.ack,...,mqtt.len,mqtt.msg_decoded_as,mqtt.msgtype,mqtt.proto_len,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_label
count,157800.000000,157800.000000,157800.000000,157800.000000,1.578000e+05,157800.0,157800.000000,157800.000000,157800.0,1.578000e+05,...,157800.000000,157800.0,157800.000000,157800.000000,157800.000000,157800.000000,157800.0,157800.0,157800.0,157800.000000
mean,0.014195,0.059848,3047.291838,3239.979778,4.046816e+04,0.0,14.715520,0.045748,0.0,7.160039e+07,...,0.419341,0.0,0.161331,0.031686,0.189506,0.031686,0.0,0.0,0.0,0.846001
std,0.149783,0.596245,11144.328203,11406.072994,1.764075e+06,0.0,229.659671,0.208938,0.0,3.101231e+08,...,3.606594,0.0,1.293453,0.354598,2.124206,0.354598,0.0,0.0,0.0,0.360949
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.000000e+00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.000000e+00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,1.000000e+00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,4.790000e+02,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.000000
max,2.000000,6.000000,65532.000000,65524.000000,7.728902e+07,0.0,83655.000000,1.000000,0.0,2.147333e+09,...,39.000000,0.0,14.000000,4.000000,24.000000,4.000000,0.0,0.0,0.0,1.000000


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157800 entries, 0 to 157799
Data columns (total 63 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   frame.time                 157800 non-null  object 
 1   ip.src_host                157800 non-null  object 
 2   ip.dst_host                157800 non-null  object 
 3   arp.dst.proto_ipv4         157800 non-null  object 
 4   arp.opcode                 157800 non-null  float64
 5   arp.hw.size                157800 non-null  float64
 6   arp.src.proto_ipv4         157800 non-null  object 
 7   icmp.checksum              157800 non-null  float64
 8   icmp.seq_le                157800 non-null  float64
 9   icmp.transmit_timestamp    157800 non-null  float64
 10  icmp.unused                157800 non-null  float64
 11  http.file_data             157800 non-null  object 
 12  http.content_length        157800 non-null  float64
 13  http.request.uri.query     15

Exploring the dataset - 

In the datasets directory there are 3 sub-directories:
1. normal_traffic - contains non-attacked traffic of the different devices in the network
2. attack_traffic - contains traffic with the attacks performed by the attacking machines 
3. dataset_ML_DL - contains sample dataset for ML and a 10 times larger one for DNN

The data is described in the research paper in the same directory as this notebook

ML-EdgeIIoT-dataset.csv

Working with the dataset designated for ML researching - ML-EdgeIIoT-dataset.csv.
This csv contains a dataset where almost 85% of it is attacks and 15% are is normal traffic.
The attacks are also divided in groups so we have two tasks binary classification and multilabel classification (for binary clf the dataset is not exactly balanced).

1. There are no empty rows or values
2. In the dataset the information about 9 protocols is saved (arp, ip, tcp, udp, icmp, http, mqtt, mbtcp, dns)
3. There are some problems in the data - for example some columns don't contain the same data type for all it's values
4. Some data is string so it needs to be converted to another type of data (ip.src_host, ip.dst_host, arp.src.proto_ipv4, http.file_data etc..)

In [57]:
len(df[df["http.file_data"] != 0][df["http.file_data"] != "0.0"][df["http.file_data"] != "0"]['http.file_data'].unique())


494

### Easy string converting

In [70]:
def convert_to_float(df):
    """Converts "0", "0.0", or 0 to float(0.0) in a DataFrame"""
    cond = (df == "0") | (df == 0) | (df == "0.0")
    return df.where(~cond, float(0.0))

df = convert_to_float(df.copy())

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157800 entries, 0 to 157799
Data columns (total 63 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   frame.time                 157800 non-null  object 
 1   ip.src_host                157800 non-null  object 
 2   ip.dst_host                157800 non-null  object 
 3   arp.dst.proto_ipv4         157800 non-null  object 
 4   arp.opcode                 157800 non-null  float64
 5   arp.hw.size                157800 non-null  float64
 6   arp.src.proto_ipv4         157800 non-null  object 
 7   icmp.checksum              157800 non-null  float64
 8   icmp.seq_le                157800 non-null  float64
 9   icmp.transmit_timestamp    157800 non-null  float64
 10  icmp.unused                157800 non-null  float64
 11  http.file_data             157800 non-null  object 
 12  http.content_length        157800 non-null  float64
 13  http.request.uri.query     15

## checkiing string data

1. frame.time - can be used in LSTM layer or converted as the time interval between two frames
2. ip.src_host, ip.dst_host, arp.dst_host - can be converted to an id and treated like a group assosiation or converted to one-hot array. Another option is to drop those columns as they are maybe not relevant in the verification of if a message is malicious or not

### Check the difference in string headers between non-attacking frames and attacking frames

The conclusion from all the verifications below is that HTTP is not used as a protocol of data transferring in a legitimate way in this network as the frames labeld as normal frames have empty http fields

Note that this helps us for the binary classification problem and not necessarily for the multilabel task

for multilabel classification maybe use dummy encoding

#### http.file_data

In [91]:
df[df["Attack_label"] == 1][df["http.file_data"] != 0]["http.file_data"]

34064     <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//...
34079     <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//...
34129     <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//...
34144     <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//...
34232     <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//...
                                ...                        
104840    <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//...
104873    <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//...
104883    <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//...
104898    <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//...
104906    <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//...
Name: http.file_data, Length: 9471, dtype: object

In [130]:
len(df[df["Attack_label"] == 1][df["http.file_data"] != 0]["http.file_data"].unique())

494

In [98]:
df[df["Attack_label"] == 0][df["http.file_data"] != 0]["http.file_data"]

Series([], Name: http.file_data, dtype: object)

The file_data in non-attack frames is empty and in the attacks frames have data sometimes so maybe i can convert to "there_is_data" and "there_is_no_data" (categorical values) (it's like this also for the DNN dataset)

#### http.request.uri.query

In [112]:
df[df["Attack_label"] == 0][df["http.request.uri.query"] != 0]["http.request.uri.query"]

Series([], Name: http.request.uri.query, dtype: object)

In [224]:
df[df["Attack_label"] == 1][df["http.request.uri.query"] != 0]["http.request.uri.query"].unique()

array(['id=2%20AND%205107%3D2126&Submit=Submitid=53',
       'id=2%20AND%201053%3D6910--%20kVQK&Submit=Submitid=53',
       'id=2%29%20AND%20EXTRACTVALUE%287898%2CCONCAT%280x5c%2C0x71786a7671%2C%28SELECT%20%28ELT%287898%3D7898%2C1%29%29%29%2C0x71626a7871%29%29%20AND%20%286070%3D6070&Submit=Submitid=53',
       ..., 'conteudo=http://cirt.net/rfiinc.txt?',
       'REDSYS[MYPATH][TEMPLATES]=http://cirt.net/rfiinc.txt?',
       'Codebase=http://cirt.net/rfiinc.txt?'], dtype=object)

The file_data in non-attack frames is empty and in the attacks frames have data sometimes so maybe i can convert to "there_is_data" and "there_is_no_data" (categorical values) (it's like this also for the DNN dataset)

In [113]:
df_dnn = pd.read_csv('iot_data/datasets/dataset_ML_DL/DNN-EdgeIIoT-dataset.csv')

#### http.request.method

In [116]:
df[df["Attack_label"] == 1][df["http.request.method"] != 0]["http.request.method"]

13143       GET
13204       GET
13208       GET
13228       GET
13289       GET
          ...  
104823      GET
104839    TRACE
104871      GET
104897      GET
104905      GET
Name: http.request.method, Length: 7196, dtype: object

In [119]:
df_dnn[df_dnn["Attack_label"] == 0][df_dnn["http.request.method"] != 0][df_dnn["http.request.method"] != "0.0"][df_dnn["http.request.method"] != "0"]["http.request.method"]


Series([], Name: http.request.method, dtype: object)

The file_data in non-attack frames is empty and in the attacks frames have data sometimes so maybe i can convert to "there_is_data" and "there_is_no_data" (categorical values) (it's like this also for the DNN dataset)

#### http.referer

In [229]:
df[df["Attack_label"] == 0][df["http.referer"] != 0]["http.referer"].unique()

array([], dtype=object)

In [122]:
df_dnn[df_dnn["Attack_label"] == 0][df_dnn["http.referer"] != 0][df_dnn["http.referer"] != "0.0"][df_dnn["http.referer"] != "0"]["http.referer"]


Series([], Name: http.referer, dtype: object)

The file_data in non-attack frames is empty and in the attacks frames have data sometimes so maybe i can convert to "there_is_data" and "there_is_no_data" (categorical values) (it's like this also for the DNN dataset)

#### http.request.full_uri

In [232]:
len(df[df["Attack_label"] == 0][df["http.request.full_uri"] != 0]["http.request.full_uri"].unique())

0

In [125]:
df_dnn[df_dnn["Attack_label"] == 0][df_dnn["http.request.full_uri"] != 0][df_dnn["http.request.full_uri"] != "0.0"][df_dnn["http.request.full_uri"] != "0"]["http.request.full_uri"]


Series([], Name: http.request.full_uri, dtype: object)

The file_data in non-attack frames is empty and in the attacks frames have data sometimes so maybe i can convert to "there_is_data" and "there_is_no_data" (categorical values) (it's like this also for the DNN dataset)

####  http.request.version

In [234]:
df[df["Attack_label"] == 0][df["http.request.version"] != 0]["http.request.version"].unique()

array([], dtype=object)

In [127]:
df_dnn[df_dnn["Attack_label"] == 0][df_dnn["http.request.version"] != 0][df_dnn["http.request.version"] != "0.0"][df_dnn["http.request.version"] != "0"]["http.request.version"]


Series([], Name: http.request.version, dtype: object)

The file_data in non-attack frames is empty and in the attacks frames have data sometimes so maybe i can convert to "there_is_data" and "there_is_no_data" (categorical values) (it's like this also for the DNN dataset)

#### tcp.options

In [239]:
df[df["Attack_label"] == 1][df["tcp.options"] != 0]["tcp.options"].unique()

array(['67.0', '5355.0', '5353.0', ..., '0101080a9e4e88c59ac0f5fb',
       '0101080a9ac0f5fa9e4e88bc', '0101080a9ac0f6019e4e88c1'],
      dtype=object)

This is an hex number describing the tcp.options field in the transported frames.
I can either convert it to a number and normalize it or decode it and create categorical values

#### tcp.payload

In [155]:
def count_unique_ratio(df, field_name, attack=1):
    # Count the number of unique values and total rows
    unique_count = df[df["Attack_label"] == attack][df[field_name] != 0][field_name].nunique()
    total_rows = len(df)

    # Calculate and return the ratio
    return unique_count / total_rows


In [243]:
df[df["Attack_label"] == 0][df["tcp.payload"] != 0]["tcp.payload"].unique()

array(['20020000', '100c00044d5154540402003c0000', 'e000', ...,
       '3027001854656d70657261747572655f616e645f48756d696469747932332e39352037352e31320d0a',
       '3027001854656d70657261747572655f616e645f48756d696469747932332e39382037352e31360d0a',
       '3027001854656d70657261747572655f616e645f48756d696469747932342e30312037352e32310d0a'],
      dtype=object)

In [156]:
count_unique_ratio(df, "tcp.payload", attack=1)

0.15423320659062104

In [157]:
count_unique_ratio(df, "tcp.payload", attack=0)

0.019195183776932825

There are more unique values compared to the number of rows of tcp.payload for non-attack frames

#### tcp.srcport

In [414]:
df[df["Attack_label"] == 1][df["tcp.srcport"] != 0]["tcp.srcport"].unique()

array([33047., 65535., 44965., ..., 63006.,  3416.,  3922.])

In [189]:
counter = 0

for index, row in df[df["Attack_label"] == 1].iterrows():
    if type(row["tcp.srcport"]) != float:
        try:
            row["tcp.srcport"] = float(row["tcp.srcport"])
        except:
            counter+=1
            print(f"index: {index}, data: {row['tcp.srcport']}")
print(counter)

index: 1615673, data: DESKTOP-UHF0SF2
index: 1615675, data: DESKTOP-UHF0SF2.local
index: 1615676, data: DESKTOP-UHF0SF2.local
index: 1615678, data: DESKTOP-UHF0SF2
index: 1615683, data: DESKTOP-UHF0SF2.local
index: 1615684, data: DESKTOP-UHF0SF2
index: 1615685, data: DESKTOP-UHF0SF2.local
index: 1615686, data: DESKTOP-UHF0SF2
index: 1615708, data: _googlecast._tcp.local
index: 1615709, data: _googlecast._tcp.local
index: 1615710, data: _googlecast._tcp.local
index: 1615711, data: _googlecast._tcp.local
index: 1615713, data: _googlecast._tcp.local
index: 1615714, data: _googlecast._tcp.local
index: 1615715, data: _googlecast._tcp.local
index: 1615716, data: _googlecast._tcp.local
index: 1615717, data: _googlecast._tcp.local
index: 1615718, data: _googlecast._tcp.local
index: 1615719, data: _googlecast._tcp.local
index: 1615720, data: _googlecast._tcp.local
index: 1615729, data: DESKTOP-UHF0SF2
index: 1615730, data: DESKTOP-UHF0SF2
index: 1615733, data: DESKTOP-UHF0SF2.local
index: 16157

367


for some reason the 367 rows in the MITM data are corrupted as the tcp source port has wrong data type and some fields are filled in tcp and in udp...

concider dropping those rows

#### dns.qry.name.len

In [196]:
counter = 0

for index, row in df[df["Attack_label"] == 1].iterrows():
    if type(row["dns.qry.name.len"]) != float:
        try:
            print(index)
            row["dns.qry.name.len"] = float(row["dns.qry.name.len"])
        except:
            counter+=1
            print(f"index: {index}, data: {row['dns.qry.name.len']}")
print(counter)

0


#### mqtt.conack.flags

In [270]:
df[df["Attack_label"] == 1][df["mqtt.conack.flags"] != 0][df["mqtt.conack.flags"] != "0x00000000"]["mqtt.conack.flags"]

Series([], Name: mqtt.conack.flags, dtype: object)

In [206]:
df["mqtt.conack.flags"].unique()

array([0.0], dtype=object)

In [204]:
def convert_to_float2(df):
    """Converts "0", "0.0", or 0 to float(0.0) in a DataFrame"""
    cond = (df == "0x00000000")
    return df.where(~cond, float(0.0))

df = convert_to_float2(df.copy())

#### mqtt.msg

In [277]:
len(df[df["Attack_label"] == 0][df["mqtt.msg"] != 0]["mqtt.msg"].unique())

115

All attack frames have mqtt msg empty so maybe convert it to categorical value

#### mqtt.protoname

In [279]:
df[df["Attack_label"] == 1][df["mqtt.protoname"] != 0]["mqtt.protoname"].unique()

array([], dtype=object)

All attack frames have mqtt protoname empty so maybe convert it to categorical value

#### mqtt.topic

In [280]:
df[df["Attack_label"] == 0][df["mqtt.topic"] != 0]["mqtt.topic"].unique()

array(['Temperature_and_Humidity'], dtype=object)

All attack frames have mqtt topic empty so maybe convert it to categorical value

## Modifying string objects to integers and floats

In [262]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


def convert_to_numeric_all_columns(df):
 
    # Iterate over each column
    for col in df.columns:
        # Create a copy of the column to avoid modifying the original DataFrame
        converted_column = df[col].copy()

        # Try converting string values to floats, ignoring errors for non-numeric values
        for i in range(len(converted_column)):
            try:
                converted_column.iloc[i] = float(converted_column.iloc[i])
            except ValueError:
                pass

        # Assign the modified column back to the DataFrame
        df[col] = converted_column

    # Return the modified DataFrame
    return df


def convert_to_data_groups(df, column_name):
    df['column_name'] = df[column_name].notna() & (df[column_name] != 0)
    df['column_name'] = df['column_name'].astype(float)
    return df


def convert_to_one_hot(df, column_name):
    categories = df[column_name].unique()
    df_encoded = pd.get_dummies(df, columns=[column_name], prefix=column_name)
    return df_encoded


def convert_to_hex(df, column_name):
    indexes_to_convert = []

    for index, row in df.iterrows():

        if type(row[column_name]) is str:
            indexes_to_convert.append(index)
    df[column_name][indexes_to_convert] = pd.to_numeric(df[column_name][indexes_to_convert].apply(lambda x: int(x, 16)), errors='coerce')


def drop_parse_error_rows(df, column_name):
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
    df = df.dropna(subset=[column_name])
    return df




steps = [("convert_to_numeric", convert_to_numeric_all_columns)]

#### Conclusions - 

In [ ]:
# binary_groups - columns converted to "has data" and "doesn't have data" (as only attack frames contain data)
# one_hot_columns - columns to be converted to one hot array
# hex_to_int - columns containig string hex value
# to_drop - columns to drop from the df

# Note that frame time and ip data will be dropped in all models except from LSTM model
binary_groups = ["http.file_data", "http.request.uri.query", "http.request.full_uri", "mqtt.conack.flags",
                 "mqtt.protoname", "mqtt.topic", "arp.hw.size", "mqtt.conflags", "mqtt.proto_len",]

one_hot_columns = ["http.request.method", "http.referer", "http.request.version", "arp.opcode",
                   "dns.qry.qu", "dns.retransmission", "mqtt.hdrflags", "mqtt.msgtype"]

hex_to_int = ["tcp.options"]

to_drop = ["tcp.payload", "dns.qry.name.len", "mqtt.msg", "icmp.transmit_timestamp", "icmp.unused",
           "http.tls_port", "dns.qry.type", "dns.retransmit_request_in", "mqtt.msg_decoded_as",
           "mbtcp.len", "mbtcp.trans_id", "mbtcp.unit_id"]

to_normalize = ["icmp.checksum", "icmp.seq_le", "http.content_length", "tcp.ack_raw", "tcp.ack",
                "tcp.checksum", "tcp.dstport", "tcp.flags", "tcp.len", "tcp.options",
                "tcp.seq", "tcp.srcport", "udp.port", "udp.stream", "udp.time_delta",
                "dns.qry.name", "mqtt.len", "mqtt.topic_len", "mqtt.ver"]

to_drop_non_lstm_and_no_flow_data = ["frame.time", "ip.src_host", "ip.dst_host", 
                                     "arp.src.proto_ipv4", "arp.dst.proto_ipv4"]